In [1]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
BigLotsAccount_id="act_271491453638620"
import json
import os
import pandas as pd
import datetime
import numpy as np
import time
token=json.load(open("/home/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']


In [2]:
# create new adueince and add the new users to the audiecne, 1 step
# Please note the change of the CustomAudience.Schema (row 41 based on the user list format)
def creatfbaudience(AudienceName,AudienceDescription,df,ACCESS_TOKEN,accountid,df_use_col_name,fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v8.0')
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    print("Audience ID:",fbname)
    audience = CustomAudience(fbname)
    users = list(df[df_use_col_name])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(fb_schema, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)


In [3]:
file_test=pd.read_table("/home/sharefolder/biglots_data/2020_by_weeks/MediaStorm_2020-08-22/MediaStormMasterWeekly20200825-113108-479.txt",
                       nrows=None,sep="|",usecols=['email_address_hash'])
print(file_test.shape)
file_test_100000=file_test.head(100000)
file_test_remaining=file_test.iloc[100000:,]
file_test_100000['seg']="test_100000"
file_test_100000.to_csv("./file_test_100000.csv")
list_all_seg_sliced_files=['./file_test_100000.csv']

(171396, 1)


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
for file in list_all_seg_sliced_files:
    audience_name = os.path.basename(file).split(".")[0]
    description=os.path.basename(file).split(".")[0]+"_"+str(datetime.datetime.now().date())
    df = pd.read_csv(file,dtype=str)
    creatfbaudience(AudienceName=audience_name,
                    AudienceDescription=description,
                    df=df,
                    df_use_col_name='email_address_hash',
                    ACCESS_TOKEN=token,
                    accountid=BigLotsAccount_id)

Audience ID: 23845186351820265
2020-08-31 16:01:27.472714 file_test_100000 uploaded 100000


In [5]:
list_remove_ids=file_test_100000['email_address_hash'].tolist()[:30000]
list_add_ids=file_test_remaining['email_address_hash'].tolist()

print(len(list_remove_ids))
print(len(list_add_ids))

30000
71396


In [6]:
audience = CustomAudience(23845186351820265)
audience

<CustomAudience> {
    "id": 23845186351820265
}

In [9]:
chunck_size=1000

for i in range(int(np.ceil(len(list_remove_ids)/chunck_size))):
    list_chunck_to_delete=list_remove_ids[i*chunck_size:(i+1)*chunck_size]
    audience.remove_users(schema=CustomAudience.Schema.email_hash, 
                    users=list_chunck_to_delete)

    time.sleep(1)


In [10]:
audience.add_users(schema=CustomAudience.Schema.email_hash, 
                    users=list_add_ids)
